# Data Centre Modelling
## Ryan Jenkinson

### Imports

In [ ]:
from dataclasses import dataclass
from typing import Literal

import numpy as np
import pandas as pd
import plotly.express as px
import pypsa
import xarray as xr

In [ ]:
np.random.seed(42)  # for reproducibility

### Constants

In [ ]:
@dataclass
class Costs:
    capital_cost: float # per MW per year
    marginal_cost: float # per MWh

# Maximum capacity assumptions

In [ ]:
# Set to None to ensure the model is just cost optimal without any maximum capacities
# Have defaulted to reasonable maximum capacities based on GB build out, but model chooses much less than this depending on DC size
WIND_MAX_CAPACITY_MW = 1400
SOLAR_MAX_CAPACITY_MW = 600
GAS_MAX_CAPACITY_MW = 77 # Gets a ~95% renewable scenario
GAS_MAX_CAPACITY_MW = 100 # To match the BEIS reference model, can also set to None to see unrestricted case
BESS_MAX_CAPACITY_MW, BESS_MAX_CAPACITY_MWH = 300, 600

### Cost Assumptions

In [ ]:
## TODO: Sensitivity analysis for costs
cost_scenario: Literal["low", "medium", "high"] = "medium"
# Rates of inflation taken from Bank of England Calculator: https://www.bankofengland.co.uk/monetary-policy/inflation/inflation-calculator
POUND_2021_TO_POUND_2025 = 1.15 # DESNZ costs are in 2021 prices, convert to 2025 prices
POUND_2012_TO_POUND_2025 = 1.45 # SMR energy 2costs are in 2012 prices, convert to 2025 prices

## Wind costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 project commencement year, Offshore Wind reference model
KW_TO_MW = 1000
wind_pre_development_costs_gbp_per_mw = {
    "low": 60 * KW_TO_MW,
    "medium": 130 * KW_TO_MW,
    "high": 200 * KW_TO_MW,
}
wind_construction_costs_gbp_per_mw = {
    "low": 1300 * KW_TO_MW,
    "medium": 1500 * KW_TO_MW,
    "high": 2000 * KW_TO_MW,
}
wind_infrastructure_costs_gbp = {
    "low": 56_000_000,
    "medium": 64_300_000,
    "high": 74_100_000,
}
wind_insurance = 3000  # £ per MW per year (insurance cost)

wind_capex_per_mw = (
    wind_pre_development_costs_gbp_per_mw[cost_scenario]
    + wind_construction_costs_gbp_per_mw[cost_scenario]
    + wind_insurance
)  # £ per MW
wind_lifetime_years = 30
wind_discount_rate = 0.07
annuitised_wind_cost = (wind_capex_per_mw * wind_discount_rate) / (
    1 - (1 + wind_discount_rate) ** -wind_lifetime_years
)
wind_fixed_opex = 43300  # £ per MW per year (fixed O&M)
wind_variable_operation_and_management_per_mwh = (
    1  # £ per MWh_elec (non-fuel variable O&M)
)

wind_connection_and_use_of_system_charges = 44800  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
WIND_COSTS = Costs(
    capital_cost=(annuitised_wind_cost + wind_fixed_opex)
    * POUND_2021_TO_POUND_2025,  # £ per MW per year
    marginal_cost=wind_variable_operation_and_management_per_mwh
    * POUND_2021_TO_POUND_2025,  # £ per MWh
)

## Solar costs
# https://www.gov.uk/government/publications/electricity-generation-costs-2023
# Using 2025 project commencement year, "Large-scale Solar" reference model
KW_TO_MW = 1000
solar_pre_development_costs_gbp_per_mw = {
    "low": 10 * KW_TO_MW,
    "medium": 50 * KW_TO_MW,
    "high": 110 * KW_TO_MW,
}
solar_construction_costs_gbp_per_mw = {
    "low": 300 * KW_TO_MW,
    "medium": 400 * KW_TO_MW,
    "high": 400 * KW_TO_MW,
}
solar_infrastructure_costs_gbp = {
    "low": 1_300_000,
    "medium": 1_400_000,
    "high": 1_500_000,
}
solar_insurance = 2000  # £ per MW per year (insurance cost)

solar_capex_per_mw = (
    solar_pre_development_costs_gbp_per_mw[cost_scenario]
    + solar_construction_costs_gbp_per_mw[cost_scenario]
    + solar_insurance
)  # £ per MW
solar_lifetime_years = 35
solar_discount_rate = 0.07
annuitised_solar_cost = (solar_capex_per_mw * solar_discount_rate) / (
    1 - (1 + solar_discount_rate) ** -solar_lifetime_years
)
solar_fixed_opex = 6000  # £ per MW per year (fixed O&M)
solar_variable_operation_and_management_mwh = (
    0  # £ per MWh_elec (non-fuel variable O&M)
)

solar_connection_and_use_of_system_charges = 1300  # £ per MW per year (grid connection costs) [NOTE: Not used in model as a microgrid]
SOLAR_COSTS = Costs(
    capital_cost=(annuitised_solar_cost + solar_fixed_opex)
    * POUND_2021_TO_POUND_2025,  # £ per MW per year
    marginal_cost=solar_variable_operation_and_management_mwh
    * POUND_2021_TO_POUND_2025,  # £ per MWh
)

## Gas costs
# Assume OCGT (Open Cycle Gas Turbine) as the reference model, 2025 project commencement year
# CCGTs are an alternative for continuous, baseload power generation but they have a much longer start-up time and a significantly larger and more complex footprint so we use OCGT costs here
# Operating Hours: 500 hr vs. 2000 hr --> 500hr is more akin to "backup" generator. 2000hr is equivalent to a standard 8 hour workday every weekday. So we use the 500hr variant
# OCGT 100MW 500 hr -> Logical starting point. Most data centres use 10 x 10MW backups so modelling as a 100MW is a good proxy
# OCGT 299MW 500 hr --> common to model even larger data centres as after 300MW there is much more complicated regulations
# Chosen reference model: OCGT 500hr 100MW
# To use a different reference model, see:
# https://www.gov.uk/government/publications/electricity-generation-costs-2023

gas_reference_model: Literal["100MW", "299MW"] = (
    "100MW"  # Reference model for gas turbine costs
)

gas_fuel_efficiency = {
    "100MW": 0.35,  # Efficiency of the 100MW OCGT (MWh_elec / MWh_fuel)
    "299MW": 0.34,  # Efficiency of the 299MW OCGT (MWh_elec / MWh_fuel)
}
gas_pre_development_costs_gbp_per_mw = {
    "100MW": {"low": 80 * KW_TO_MW, "medium": 90 * KW_TO_MW, "high": 110 * KW_TO_MW},
    "299MW": {"low": 30 * KW_TO_MW, "medium": 40 * KW_TO_MW, "high": 40 * KW_TO_MW},
}
gas_construction_costs_gbp_per_mw = {
    "100MW": {"low": 600 * KW_TO_MW, "medium": 700 * KW_TO_MW, "high": 900 * KW_TO_MW},
    "299MW": {"low": 300 * KW_TO_MW, "medium": 500 * KW_TO_MW, "high": 800 * KW_TO_MW},
}
gas_infrastructure_costs_gbp = {
    "100MW": {"low": 7_200_000, "medium": 14_400_000, "high": 28_700_000},
    "299MW": {"low": 7_800_000, "medium": 15_500_000, "high": 31_000_000},
}
gas_insurance = {
    "100MW": 2900,  # £ per MW per year (insurance cost for 100MW OCGT)
    "299MW": 1800,  # £ per MW per year (insurance cost for 299MW OCGT)
}
gas_connection_use_of_system_charges = 2700  # NOTE: Not used in model as a microgrid

gas_capex_per_mw = (
    gas_pre_development_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_construction_costs_gbp_per_mw[gas_reference_model][cost_scenario]
    + gas_insurance[gas_reference_model]
)
gas_lifetime_years = 25
gas_discount_rate = 0.07
annuitised_gas_cost = (gas_capex_per_mw * gas_discount_rate) / (
    1 - (1 + gas_discount_rate) ** -gas_lifetime_years
)
gas_fixed_opex = {
    "100MW": 11300,  # £ per MW per year (fixed O&M for 100MW OCGT)
    "299MW": 7300,  # £ per MW per year (fixed O&M for 299MW OCGT)
}[gas_reference_model]  # Fixed O&M cost based on the reference model
gas_fuel_price_pence_per_therm = 80  # Taken from 2025 September Gas Price Futures gas price
PENCE_PER_THERM_TO_GBP_PER_MWH = 2.93071
gas_fuel_price_mwh = gas_fuel_price_pence_per_therm / PENCE_PER_THERM_TO_GBP_PER_MWH
gas_variable_operation_and_management_mwh = {
    "100MW": 2,  # £ per MWh_elec (non-fuel variable O&M for 100MW OCGT)
    "299MW": 1,  # £ per MWh_elec (non-fuel variable O&M for 299MW OCGT)
}[gas_reference_model]
gas_marginal_cost_fuel = (
    gas_fuel_price_mwh / gas_fuel_efficiency[gas_reference_model]
)  # £ per MWh_elec
GAS_COSTS = Costs(
    capital_cost=(annuitised_gas_cost + gas_fixed_opex)
    * POUND_2021_TO_POUND_2025,  # £ per MW per year
    marginal_cost=gas_marginal_cost_fuel
    + (
        gas_variable_operation_and_management_mwh * POUND_2021_TO_POUND_2025
    ),  # £ per MWh_elec
)

## BESS (Battery Energy Storage System) costs
# https://modoenergy.com/research/gb-november-2024-research-roundup-battery-energy-storage-capex-long-duration-carbon-emmisions-connections-reform-recycling-clean-power-2030
# https://haush.co.uk/renewable-energy-storage-technologies/
# All BESS costs are in £(2025) prices from research

# For a typical grid-scale BESS project in Great Britain, the total capital expenditure currently averages around £580,000 per megawatt (MW).
# The battery system itself, encompassing the battery modules, racks, containers, heating, ventilation, and air conditioning (HVAC),
# and the power conversion system (PCS), constitutes the largest portion of this, accounting for approximately 46% of the total project cost.
# This puts the battery system capex in the region of £267,000 per MW.
# On a per-unit-of-energy basis, the capex for utility-scale lithium-ion batteries typically ranges from £150 to £200 per kilowatt-hour (kWh).
bess_energy_capex_per_mwh = {
    "low": 150_000,  # £ per MWh
    "medium": 175_000,  # £ per MWh
    "high": 200_000,  # £ per MWh
}
bess_power_capex_per_mw = {
    "medium": 267_000,  # £ per MW, from Modo figure but discounting the energy storage part which is calculated above
}
bess_power_capex_per_mw["low"] = bess_power_capex_per_mw["medium"] * 0.9
bess_power_capex_per_mw["high"] = bess_power_capex_per_mw["medium"] * 1.1
bess_lifetime_years = 15
bess_discount_rate = 0.07

annuitised_bess_energy_cost = (
    bess_energy_capex_per_mwh[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
bess_capital_cost_energy = (
    annuitised_bess_energy_cost  # Cost for energy capacity (Store)
)

# Assume all OPEX cost of £2-8 per kWh/year is on energy part, not links, as per source. Includes insurance etc
battery_opex_cost_per_mwh = {
    "low": 0.002,  # £ per MWh/year
    "medium": 0.005,  # £ per MWh/year
    "high": 0.008,  # £ per MWh/year
}

annuitised_bess_power_cost = (
    bess_power_capex_per_mw[cost_scenario] * bess_discount_rate
) / (1 - (1 + bess_discount_rate) ** -bess_lifetime_years)
# bess_fixed_opex_per_mw_power = 20000  # Fixed OPEX for power conversion part
bess_capital_cost_power = annuitised_bess_power_cost  # + bess_fixed_opex_per_mw_power  # Cost for power capacity (Links)

bess_round_trip_efficiency = 0.87
# bess_marginal_cost_links = 0.001  # Marginal cost for charging/discharging
bess_marginal_cost_links = 0  # Assume no marginal cost for using the links, just capital cost. All OPEX is in the energy storage part
# Standing loss could be modelled in PyPSA, but after a sensitivity analysis from 0 to ~30 kWh per MWh (0 - 0.2%) it was deemed to not have a significant effect on results
# So for simplicity we do not include it here to avoid an additional modelling parameter

BESS_COSTS = Costs(
    capital_cost=bess_capital_cost_energy,  # £ per MWh per year
    marginal_cost=battery_opex_cost_per_mwh[cost_scenario],  # £ per MWh_elec
)
BESS_LINK_COSTS = Costs(
    capital_cost=bess_capital_cost_power,  # £ per MW per year
    marginal_cost=bess_marginal_cost_links,  # £ per MWh_elec
)

# Cannot add in infrastructure costs to calculations above as they are just £ total not depending on load e.g per MW in the DESNZ model
# BESS infrastructure costs are given in the capex/opex values above so already included. Do not need to reinclude them here
# Note: These would become less of a percentage of the total cost for larger data centres (!!)
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS = (
    wind_infrastructure_costs_gbp[cost_scenario]
    * wind_discount_rate
    * POUND_2021_TO_POUND_2025
) / (1 - (1 + wind_discount_rate) ** -wind_lifetime_years) + (
    (
        gas_infrastructure_costs_gbp[gas_reference_model][cost_scenario]
        * gas_discount_rate
        * POUND_2021_TO_POUND_2025
    )
    / (1 - (1 + gas_discount_rate) ** -gas_lifetime_years)
)
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_WITH_SOLAR = (
    TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
    + (
        (
            solar_infrastructure_costs_gbp[cost_scenario]
            * solar_discount_rate
            * POUND_2021_TO_POUND_2025
        )
        / (1 - (1 + solar_discount_rate) ** -solar_lifetime_years)
    )
)

# Small Modular Reactor costs
SMR_COST_FOR_450MW_UNIT = 2.5e9  # £(2025) per unit, assuming a single unit
smr_lifetime_years = 60  # Assumed lifetime of the SMR in years, from Rolls Royce : https://gda.rolls-royce-smr.com/our-technology#:~:text=Over%20its%2060%2Dyear%20lifetime,type%20of%20waste%20is%20it%3F
smr_discount_rate = 0.07  # assumed the same as wind and gas
smr_capital_cost = (
    (SMR_COST_FOR_450MW_UNIT / 450)
    * smr_discount_rate
    / (1 - (1 + smr_discount_rate) ** -smr_lifetime_years)
)  # Annuitised capital
smr_marginal_cost = {
    # In a 2022 submission, Rolls‑Royce stated its power will cost “around £50/60 per megawatt hour” in 2012 prices - which it re-confirmed was its position in early 2025
    # https://www.theguardian.com/environment/2025/jan/15/a-viable-business-rolls-royce-banking-on-success-of-small-modular-reactors
    "low": 50 * POUND_2012_TO_POUND_2025,
    "medium": 55 * POUND_2012_TO_POUND_2025,
    "high": 60 * POUND_2012_TO_POUND_2025,
}

SMR_COSTS = Costs(
    capital_cost=smr_capital_cost,  # £ per MW per year
    marginal_cost=smr_marginal_cost[cost_scenario],  # £ per MWh_elec
)

smr_infrastructure_costs_gbp = 0  # TODO: Get any other external infrastructure costs
TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR = (
    smr_infrastructure_costs_gbp * smr_discount_rate
) / (1 - (1 + smr_discount_rate) ** -smr_lifetime_years)

In [ ]:
DATA_CENTRE_LOAD_MW = 120  # MW, constant demand including PUE

## Wind and Solar Profiles

Get the availabilities of wind and solar profiles

In [ ]:
REFERENCE_YEAR: Literal["2019", "2023"] = (
    "2023"  # Reference year for the model, using most recent available
)
# Wind Data
wind_data = xr.load_dataset(f"../data/profile_39_offshorewind-{REFERENCE_YEAR}.nc")
wind_availability_df = wind_data["profile"].to_dataframe().reset_index()
wind_availability_pu = wind_availability_df[wind_availability_df["bus"] == "GB2 0"].set_index("time")[
    "profile"
].clip(0, 1).reset_index(drop=True)
wind_capacity_factor = (
    0.61  # Target capacity factor for the wind farm (Source: DESNZ, similar to actuals)
)
wind_availability_pu = wind_capacity_factor * (
    wind_availability_pu / wind_availability_pu.mean()
)
wind_availability_pu = wind_availability_pu.to_numpy()

# Solar Data
solar_profile = xr.load_dataset(f"../data/profile_39_solar-{REFERENCE_YEAR}.nc")
solar_availability_df = solar_profile["profile"].to_dataframe().reset_index()
solar_availability_pu = solar_availability_df[solar_availability_df.bus == "GB2 0"].set_index("time")[
    "profile"
].clip(0, 1).reset_index(drop=True)
solar_capacity_factor = (
    0.11  # Target capacity factor for the solar (Source: DESNZ, similar to actuals)
)
solar_availability_pu = solar_capacity_factor * (
    solar_availability_pu / solar_availability_pu.mean()
)
solar_availability_pu = solar_availability_pu.to_numpy()

# Plot rolling hour availability of wind and solar on the same chart
availability_df = pd.DataFrame(
    {
        "Wind Availability (pu)": wind_availability_pu,
        "Solar Availability (pu)": solar_availability_pu,
    }
)
# plot the target capacity factors as horizontal lines
fig = px.line(
    availability_df.rolling(window=24).mean(),
    title="Rolling 24 Hour Average Availability of Wind and Solar",
    labels={"index": "Hour", "value": "Availability (pu)"},
).add_hline(
    y=wind_capacity_factor,
    line_dash="dash",
    line_color="lightgreen",
    annotation_text="Target Wind Capacity Factor",
    annotation_position="top left",
).add_hline(
    y=solar_capacity_factor,
    line_dash="dash",
    line_color="gold",
    annotation_text="Target Solar Capacity Factor",
    annotation_position="top left",
)
fig.show()

### Model

In [ ]:
MODEL_TIME_FREQUENCY = pd.Timedelta("1h")  # Time frequency for the model
GENERATION_TYPES: Literal["wind", "solar", "gas", "bess", "smr"] = (
    "offshore_wind",
    "gas",
    "bess",
)
def create_model(data_centre_load_mw: float, generation_types=GENERATION_TYPES, **solar_kwargs: dict):
    # Define simulation period: a full year, hourly
    snapshots = pd.date_range(
        start=f"{REFERENCE_YEAR}-01-01 00:00", end=f"{REFERENCE_YEAR}-12-31 23:00", freq=MODEL_TIME_FREQUENCY
    )
    n = pypsa.Network(snapshots=snapshots)

    # --- DEFINE CARRIERS ---
    n.add("Carrier", "AC")  # For AC buses
    n.add("Carrier", "gas", co2_emissions=0.185)  # tCO2/MWh_th, typical for natural gas
    n.add("Carrier", "offshore_wind")
    n.add("Carrier", "battery")  # Carrier for battery components

    # --- DEFINE BUSES ---
    n.add("Bus", "electricity_bus", carrier="AC")
    if "bess" in generation_types:
        n.add("Bus", "bess_connection_bus", carrier="AC")

    # --- DEFINE LOAD ---
    # Load (Data Centre): The data centre is modelled as a load with a constant power demand.
    constant_demand_profile = pd.Series(data_centre_load_mw, index=n.snapshots)
    n.add("Load", "data_centre_load", bus="electricity_bus", p_set=constant_demand_profile)

    # --- DEFINE GENERATORS ---
    # Generator (Offshore Wind Farm): The offshore wind farm's capacity is extendable.
    if "offshore_wind" in generation_types:
        n.add(
            "Generator",
            "offshore_wind_farm",
            bus="electricity_bus",
            carrier="offshore_wind",
            p_nom_extendable=True,  # Allow capacity to be extended. To cover what we need to for the data centre load
            capital_cost=WIND_COSTS.capital_cost,
            marginal_cost=WIND_COSTS.marginal_cost,
            p_max_pu=pd.Series(wind_availability_pu, index=n.snapshots),
            p_nom_max=WIND_MAX_CAPACITY_MW
        )

    if "solar" in generation_types:
        n.add(
            "Generator",
            "solar",
            bus="electricity_bus",
            carrier="AC",
            p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
            committable=False,
            capital_cost=SOLAR_COSTS.capital_cost,
            marginal_cost=SOLAR_COSTS.marginal_cost,
            p_max_pu=pd.Series(solar_availability_pu, index=n.snapshots),
            p_nom_max=SOLAR_MAX_CAPACITY_MW,
            **solar_kwargs
        )

    # Generator (Gas Turbine Backup): The gas turbine provides backup, and its capacity is also extendable.
    if "gas" in generation_types:
        n.add(
            "Generator",
            "gas_turbine",
            bus="electricity_bus",
            carrier="gas",
            p_nom_extendable=True,  # Allow capacity to be extended to cover the data centre load
            committable=False,
            capital_cost=GAS_COSTS.capital_cost,
            marginal_cost=GAS_COSTS.marginal_cost,
            p_nom_max=GAS_MAX_CAPACITY_MW
        )

    # --- DEFINE STORAGE (BESS) ---
    if "bess" in generation_types:
        # Store component representing the energy capacity of the BESS, and each of the links for charging and discharging.
        n.add(
            "Store",
            "bess_storage",
            bus="bess_connection_bus",
            carrier="battery",
            e_nom_extendable=True,
            e_cyclic=True,
            capital_cost=BESS_COSTS.capital_cost,
            e_nom_max=BESS_MAX_CAPACITY_MWH,
        )
        # Since eff_rte = eff_charge * eff_discharge, assume that eff_charge = eff_discharge [= sqrt(eff_rte)] so equally split efficiency losses
        n.add(
            "Link",
            "bess_charger",
            bus0="electricity_bus",
            bus1="bess_connection_bus",
            carrier="battery",
            p_nom_extendable=True,
            capital_cost=BESS_LINK_COSTS.capital_cost,
            efficiency=bess_round_trip_efficiency**0.5,
            marginal_cost=BESS_LINK_COSTS.marginal_cost,
            p_nom_max=BESS_MAX_CAPACITY_MW
        )
        n.add(
            "Link",
            "bess_discharger",
            bus0="bess_connection_bus",
            bus1="electricity_bus",
            carrier="battery",
            p_nom_extendable=True,
            capital_cost=0,  # Assuming power CAPEX is fully on charger link, or split if preferred
            efficiency=bess_round_trip_efficiency**0.5,
            marginal_cost=BESS_LINK_COSTS.marginal_cost,
            p_nom_max=BESS_MAX_CAPACITY_MW
        )

    if "smr" in generation_types:
        n.add(
            "Generator",
            "smr",
            bus="electricity_bus",
            carrier="AC",
            p_nom_extendable=True,
            committable=False,
            capital_cost=SMR_COSTS.capital_cost,
            marginal_cost=SMR_COSTS.marginal_cost,
        )

    # --- CONSISTENCY CHECK & SOLVE ---

    # Perform consistency check
    try:
        n.consistency_check()
        print("Network consistency check passed before solving.")
    except Exception as e:
        print(f"Consistency check error before solving: {e}")

    return n

In [ ]:
# Ensure you have a solver installed (e.g., cbc, glpk, gurobi)
SOLVER_NAME = "highs"  # or 'glpk', 'gurobi', etc.


def solve_model(n: pypsa.Network, solver_name: str = SOLVER_NAME, print_results: bool = True):
    """Solve the PyPSA network model using the specified solver.

    Parameters:
    - n: PyPSA Network object
    - solver_name: Name of the solver to use (e.g., 'cbc', 'glpk')
    - print_results: Whether to print the results after solving
    """
    try:
        # Solve the model using the optimize interface, common with linopy backend
        # The `solve_model` method handles the optimization.
        n.optimize(solver_name=solver_name)
        if not print_results:
            return n
        print("Model solved successfully.")

        # --- DISPLAY RESULTS ---
        print("\n--- Optimization Results ---")
        # Objective value is typically stored in n.model.objective after solving with n.optimize
        obj_value = n.model.objective.value
        print(f"Objective Value: {obj_value:.2f} £ (Total Cost)")

        print("\nOptimal Capacities:")
        # Check if components exist in results before trying to access them
        # Optimal capacities are in p_nom_opt (for Generators, Links) and e_nom_opt (for Stores)
        if (
            "offshore_wind_farm" in n.generators.index
            and "p_nom_opt" in n.generators.columns
        ):
            print(
                f"  Offshore Wind Farm (p_nom_opt): {n.generators.loc['offshore_wind_farm', 'p_nom_opt']:.2f} MW"
            )
        if "solar" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Solar (p_nom_opt): {n.generators.loc['solar', 'p_nom_opt']:.2f} MW"
            )
        if "gas_turbine" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  Gas Turbine (p_nom_opt): {n.generators.loc['gas_turbine', 'p_nom_opt']:.2f} MW"
            )
        if "bess_storage" in n.stores.index and "e_nom_opt" in n.stores.columns:
            print(
                f"  BESS Storage Energy Capacity (e_nom_opt): {n.stores.loc['bess_storage', 'e_nom_opt']:.2f} MWh"
            )
        if "bess_charger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Charger Power Capacity (p_nom_opt): {n.links.loc['bess_charger', 'p_nom_opt']:.2f} MW"
            )
        if "bess_discharger" in n.links.index and "p_nom_opt" in n.links.columns:
            print(
                f"  BESS Discharger Power Capacity (p_nom_opt): {n.links.loc['bess_discharger', 'p_nom_opt']:.2f} MW"
            )
        if "smr" in n.generators.index and "p_nom_opt" in n.generators.columns:
            print(
                f"  SMR (p_nom_opt): {n.generators.loc['smr', 'p_nom_opt']:.2f} MW"
            )

    except Exception as e:
        print(f"An error occurred during model solution or results processing: {e}")
        print("Please ensure you have a compatible solver installed and in your PATH.")
        print("Common solvers: cbc, highs. You might need to install them.")
        print(
            "Also ensure Linopy (PyPSA's modeling backend) is installed and functioning correctly."
        )
    return n

In [ ]:
def get_optimal_capacities(n: pypsa.Network):
    """Get the optimal capacities of the network components after solving."""
    optimal_capacities = {
        "offshore_wind_farm": n.generators.loc["offshore_wind_farm", "p_nom_opt"],
        "gas_turbine": n.generators.loc["gas_turbine", "p_nom_opt"],
        "bess_storage": n.stores.loc["bess_storage", "e_nom_opt"],
        "bess_charger": n.links.loc["bess_charger", "p_nom_opt"],
        "bess_discharger": n.links.loc["bess_discharger", "p_nom_opt"],
    }
    return optimal_capacities


def get_asset_expenditure(n: pypsa.Network):
    stats = n.statistics()
    return {
        "gas": (
            stats.loc[("Generator", "gas"), "Capital Expenditure"]
            + stats.loc[("Generator", "gas"), "Operational Expenditure"]
        ),
        "offshore_wind": (
            stats.loc[("Generator", "offshore_wind"), "Capital Expenditure"]
            + stats.loc[("Generator", "offshore_wind"), "Operational Expenditure"]
        ),
        "battery": (
            stats.loc[("Link", "battery"), "Capital Expenditure"]
            + stats.loc[("Link", "battery"), "Operational Expenditure"]
            + stats.loc[("Store", "battery"), "Capital Expenditure"]
            + stats.loc[("Store", "battery"), "Operational Expenditure"]
        ),
    }

# Run modelling scenarios

In [ ]:
scenarios = {
    "smr_only": ("smr"),
    "wind_bess_gas": ("offshore_wind", "bess", "gas"),
    "wind_bess_gas_solar": ("offshore_wind", "bess", "gas", "solar"),
}
scenario_labels = {
    "smr_only": "<b>Scenario 1</b> <i>(Baseline)</i>:<br>Nuclear SMR",
    "wind_bess_gas_solar": "<b>Scenario 2</b> <i>(Alternative)</i>:<br>Wind+Solar+BESS+Gas Microgrid",
    "wind_bess_gas": "<b>Scenario 3</b> <i>(Alternative)</i>:<br>Wind+BESS+Gas Microgrid",
}

scenario_networks = {}
for scenario_name, generation_types in scenarios.items():
    n = create_model(data_centre_load_mw=DATA_CENTRE_LOAD_MW, generation_types=generation_types)
    n = solve_model(n, print_results=True)
    scenario_networks[scenario_name] = n

In [ ]:
SCENARIO_TO_PLOT = "wind_bess_gas" # smr_only | wind_bess_gas | wind_bess_gas_solar

GENERATOR_NAMES_READABLE = {
    "smr": "SMR",
    "bess": "BESS",
    "offshore_wind_farm": "Offshore Wind",
    "solar": "Solar",
    "gas_turbine": "Gas Turbine",
}
COLOUR_MAP = {
    "SMR": "purple",
    "BESS": "orange",
    "Offshore Wind": "lightgreen",
    "Solar": "gold",
    "Gas Turbine": "darkgrey"
}


n = scenario_networks[SCENARIO_TO_PLOT]

generators = n.generators.index.tolist()

generator_power = n.generators_t.p
data_centre_power = n.loads_t.p["data_centre_load"]
asset_power = generator_power.copy()
asset_power["dc_load"] = data_centre_power
try:
    battery_power = n.links_t.p1["bess_charger"] - n.links_t.p1["bess_discharger"]
    asset_power["bess"] = battery_power
    generators += ["bess"]
except:
    pass


asset_energy_mwh_per_day = asset_power.groupby(pd.Grouper(freq="D")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)
asset_energy_mwh_per_month = asset_power.groupby(pd.Grouper(freq="M")).apply(lambda x: x.abs().sum().mul(
    MODEL_TIME_FREQUENCY.total_seconds() / 3600
))  # Convert power to energy (MWh)

fig = px.line(
    data_frame=asset_power.rename(columns=GENERATOR_NAMES_READABLE),
    title="Asset Power Output Over Time",
    labels={"value": "Power (MW)", "variable": "Asset"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=asset_power_output_over_time.html")
fig.show()

# For each time period in the day, create a stacked bar chart showing the amount of energy used from each of the generators to meet the data centre load
asset_energy_mwh_per_day = asset_energy_mwh_per_day.rename(columns=GENERATOR_NAMES_READABLE)
fig = px.bar(
    data_frame=asset_energy_mwh_per_day,
    x=asset_energy_mwh_per_day.index,
    y=list(set(asset_energy_mwh_per_day.columns.to_list()) - {"dc_load"}),
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Day of the Year",
    labels={"value": "Energy (MWh)", "variable": "Generator", "snapshot": "Day"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_day.index,
    y=asset_energy_mwh_per_day["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_day.html")
fig.show()

fig.update_xaxes(range=["2023-08-14", "2023-09-24"])
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_day_zoomed.html")

# Data used monthly
# Add percentage columns for each generator
for col in generators:
    asset_energy_mwh_per_month[f"{col}_pct"] = (
        asset_energy_mwh_per_month[col] / asset_energy_mwh_per_month["dc_load"] * 100
    ).round(1).astype(str) + "%"

# Melt the DataFrame for easier plotting with text labels
melted = asset_energy_mwh_per_month.reset_index().melt(
    id_vars=["snapshot"],
    value_vars=generators,
    var_name="Generator",
    value_name="Energy (MWh)",
)
# Add percentage labels
melted["Percentage"] = [
    asset_energy_mwh_per_month.loc[row["snapshot"], f"{row['Generator']}_pct"]
    for _, row in melted.iterrows()
]
melted["Generator"] = melted["Generator"].replace(GENERATOR_NAMES_READABLE)

fig = px.bar(
    melted,
    x="snapshot",
    y="Energy (MWh)",
    color="Generator",
    title="Energy Used from Each Generator to Meet Data Centre Load on Each Month of the Year",
    labels={"snapshot": "Month"},
    template="plotly_white",
    color_discrete_map=COLOUR_MAP,
    text="Percentage"
)
# Add a line for the data centre load
fig.add_scatter(
    x=asset_energy_mwh_per_month.index,
    y=asset_energy_mwh_per_month["dc_load"],
    mode="lines",
    name="Data Centre Load",
    line=dict(color="black", width=2),
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=energy_used_per_month.html")

fig.show()

In [ ]:
# Get optimised powers for each asset
# TODO: Fix ordering of generators
percentages = []
labels = []
if "smr" in generators:
    smr_opt_power = n.generators.loc["smr", "p_nom_opt"]
    smr_pct = smr_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(smr_pct)
    labels.append("<b>[SMR]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(smr_opt_power, smr_pct))
if "offshore_wind_farm" in generators:
    wind_opt_power = n.generators.loc["offshore_wind_farm", "p_nom_opt"]
    wind_pct = wind_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(wind_pct)
    labels.append("<b>[Offshore Wind]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(wind_opt_power, wind_pct))
if "solar" in generators:
    solar_opt_power = n.generators.loc["solar", "p_nom_opt"]
    solar_pct = solar_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(solar_pct)
    labels.append("<b>[Solar]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(solar_opt_power, solar_pct))
if "gas_turbine" in generators:
    gas_opt_power = n.generators.loc["gas_turbine", "p_nom_opt"]
    gas_pct = gas_opt_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(gas_pct)
    labels.append("<b>[Gas Turbine]</b><br><i>Power:</i> {:.1f} MW<br><i>({:.1f}%)</i>".format(gas_opt_power, gas_pct))
if "bess" in generators:
    bess_opt_energy = n.stores.loc["bess_storage", "e_nom_opt"]
    bess_input_power = n.links.loc["bess_charger", "p_nom_opt"]
    bess_output_power = n.links.loc["bess_discharger", "p_nom_opt"]
    bess_avg_power = (bess_input_power + bess_output_power) / 2
    bess_pct = bess_avg_power / DATA_CENTRE_LOAD_MW * 100
    percentages.append(bess_pct)
    labels.append("<b>[BESS]</b><br><i>Avg Power (Import/Export):</i> {:.1f} MW<br><i>Energy:</i> {:.1f} MWh<br><i>({:.1f}%)</i>".format(bess_avg_power, bess_opt_energy, bess_pct))

fig = px.bar(
    x=percentages,
    y=["Data Centre Load"] * len(generators),
    orientation="h",
    text=labels,
    color=generators,
    color_discrete_map={
        "offshore_wind_farm": "lightgreen",
        "gas_turbine": "darkgrey",
        "bess": "orange",
        "smr": "purple",
        "solar": "gold",
    },
    title=f"Optimised Microgrid Generation Powers as Percentage of Data Centre Load<br><i>(Assuming {DATA_CENTRE_LOAD_MW} MW DC Load)</i>",
    labels={"x": "Percentage of DC Load (%)", "y": ""},
)

# Add black outline to each bar
for trace in fig.data:
    trace.marker.line.width = 2
    trace.marker.line.color = "black"

fig.update_traces(texttemplate="%{text}", textposition="inside")
fig.update_layout(
    barmode="stack",
    xaxis_title="Percentage of DC Load (%)",
    yaxis_title="",
    template="plotly_white",
    showlegend=False,
)
fig.write_html(f"scenario={SCENARIO_TO_PLOT}_chart=optimised_generation_powers_percentage_of_load.html")
fig.show()

In [ ]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)
CARBON_EMISSION_GAS_PER_KWH = 0.184 # https://questions-statements.parliament.uk/written-questions/detail/2015-11-26/17799#:~:text=Technology%20%2F%20Fuel,0.497
KG_TO_TONNES = 0.001
carbon_emissions_annual = (
    total_gas_mwh_usage * (CARBON_EMISSION_GAS_PER_KWH * KW_TO_MW) * KG_TO_TONNES
)
print(f"Annual Carbon Emissions from Gas Turbine: {carbon_emissions_annual:,.2f} tCO2")


In [ ]:
HOURS_PER_YEAR = 8760  # Total hours in a year
NUCLEAR_PPA_COST_PER_MWH = 133  # 92.5 £2012/MWh in todays money
CARBON_FREE_ENERGY_PREMIUM_PERCENTAGE = 0  # Can assume a percentage hyperscalers would be willing to pay e.g 10%. Be conservative.


def calculate_costs(data_centre_load_mw: float):
    print(f"Data Centre Load: {load} MW")
    # Create the models - TODO: abstract to scenarios above instead of recalculating
    smr_network = create_model(
        data_centre_load_mw=data_centre_load_mw, generation_types=["smr"]
    )
    microgrid_network = create_model(
        data_centre_load_mw=data_centre_load_mw,
        generation_types=["gas", "bess", "offshore_wind"],
    )
    microgrid_network_with_solar = create_model(
        data_centre_load_mw=data_centre_load_mw,
        generation_types=["gas", "bess", "offshore_wind", "solar"],
    )
    # Solve the models
    smr_network = solve_model(smr_network, print_results=False)
    microgrid_network = solve_model(microgrid_network, print_results=False)
    microgrid_network_with_solar = solve_model(microgrid_network_with_solar, print_results=False)

    # Calculate the costs
    cost_of_running_microgrid_per_year = (
        microgrid_network.model.objective.value + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
    )
    cost_of_running_microgrid_with_solar_per_year = (
        microgrid_network_with_solar.model.objective.value
        + TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_WITH_SOLAR
    )
    # i.e 1 if up to 500MW, 2 if up to 1000 MW etc
    number_of_smrs_needed = (load // 500) + 1
    cost_of_running_with_nuclear_ppa_per_year = (
        load
        * HOURS_PER_YEAR
        * (NUCLEAR_PPA_COST_PER_MWH * (1 + CARBON_FREE_ENERGY_PREMIUM_PERCENTAGE))
    )
    cost_of_running_with_nuclear_smrs_per_year = (
        smr_network.model.objective.value
        + number_of_smrs_needed * TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR
    )  # £ per year, with the SMR being built for the purpose just of the data centre

    print(
        f"Cost of running the microgrid per year: £{cost_of_running_microgrid_per_year:,.2f}"
    )
    print(
        f"Cost of running the microgrid with solar per year: £{cost_of_running_microgrid_with_solar_per_year:,.2f}"
    )
    print(
        f"Cost of running with a purpose-built SMR per year: £{cost_of_running_with_nuclear_smrs_per_year:,.2f}"
    )
    # Calculate the percentage difference in costs
    cost_difference_percentage = (
        (
            cost_of_running_microgrid_per_year
            - cost_of_running_with_nuclear_smrs_per_year
        )
        / cost_of_running_with_nuclear_smrs_per_year
    ) * 100
    cost_difference_percentage_with_solar = (
        (
            cost_of_running_microgrid_with_solar_per_year
            - cost_of_running_with_nuclear_smrs_per_year
        )
        / cost_of_running_with_nuclear_smrs_per_year
    ) * 100
    print(f"Percentage difference in costs: {cost_difference_percentage:.2f}%")
    print(f"Percentage difference in costs (with solar): {cost_difference_percentage_with_solar:.2f}%")
    return (
        cost_difference_percentage,
        cost_difference_percentage_with_solar,
        cost_of_running_with_nuclear_smrs_per_year,
        cost_of_running_microgrid_per_year,
        cost_of_running_microgrid_with_solar_per_year,
    )

cost_difference_percentages = {}
cost_difference_percentages_with_solar = {}
cost_microgrid = {}
cost_microgrid_with_solar = {}
cost_nuclear_smr = {}

DC_LOADS = [50, 100, 200, 300, 400, 500, DATA_CENTRE_LOAD_MW]
DC_LOADS = [DATA_CENTRE_LOAD_MW]  # For just the one load case

for load in DC_LOADS:
    (
        cost_difference_percentage,
        cost_difference_percentage_with_solar,
        cost_of_running_with_nuclear_smrs_per_year,
        cost_of_running_microgrid_per_year,
        cost_of_running_microgrid_with_solar_per_year,
    ) = calculate_costs(load)
    cost_difference_percentages[load] = cost_difference_percentage
    cost_difference_percentages_with_solar[load] = cost_difference_percentage_with_solar
    cost_microgrid[load] = cost_of_running_microgrid_per_year
    cost_microgrid_with_solar[load] = cost_of_running_microgrid_with_solar_per_year
    cost_nuclear_smr[load] = cost_of_running_with_nuclear_smrs_per_year


In [ ]:
import plotly.graph_objects as go

load_scenario = DATA_CENTRE_LOAD_MW
nuclear_cost = cost_nuclear_smr[load_scenario]
microgrid_cost = cost_microgrid[load_scenario]
microgrid_with_solar_cost = cost_microgrid_with_solar[load_scenario]
cost_diff_pct = cost_difference_percentages[load_scenario]

# Bar chart data
scenarios = [
    "<b>Scenario 1</b> <i>(Baseline)</i>:<br>Nuclear SMR",
    "<b>Scenario 2</b> <i>(Alternative)</i>:<br>Wind+Solar+BESS+Gas Microgrid",
    "<b>Scenario 3</b> <i>(Alternative)</i>:<br>Wind+BESS+Gas Microgrid",
]
costs = [nuclear_cost, microgrid_with_solar_cost, microgrid_cost]
colors = ["#636EFA", "#00CC96", "#FFA07A"]

fig = go.Figure()

# Add bars with black outlines
for i, (scenario, cost, color) in enumerate(zip(scenarios, costs, colors)):
    fig.add_trace(
        go.Bar(
            x=[scenario],
            y=[cost],
            marker=dict(color=color, line=dict(color="black", width=3)),
            name=scenario,
            showlegend=False,
            width=0.5,
        )
    )

# Add downward arrow and annotation if microgrid is cheaper
fig.add_annotation(
    x=scenarios[2],
    y=microgrid_cost,
    text=f"{abs(cost_diff_pct):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)
fig.add_annotation(
    x=scenarios[1],
    y=microgrid_with_solar_cost,
    text=f"{abs(cost_difference_percentages_with_solar[load_scenario]):.1f}% lower",
    showarrow=True,
    arrowhead=2,
    arrowsize=1.5,
    arrowwidth=2,
    arrowcolor="black",
    ax=0,
    ay=-80,  # Negative value moves annotation above the bar
    font=dict(color="black", size=16),
    bgcolor="white",
    bordercolor="black",
    borderwidth=2,
)

fig.update_layout(
    title=f"Annual Cost Comparison: SMR vs Microgrid ({load} MW DC Load)",
    yaxis_title="Annual Cost (£)",
    xaxis_title="Scenario",
    template="plotly_white",
    bargap=0.5,
    yaxis=dict(tickformat=","),
)
fig.write_html(f"scenario=comparison_smr_vs_microgrid_{load_scenario}MW.html")

fig.show()

In [ ]:
import plotly.graph_objects as go

# Prepare scenario names and networks
scenarios_to_plot = ["smr_only", "wind_bess_gas_solar", "wind_bess_gas"]

# Collect CAPEX and OPEX for each scenario
cost_breakdown = {}
for scenario in scenarios_to_plot:
    n_scenario = scenario_networks[scenario]
    stats = n_scenario.statistics()
    capex = stats.loc[(slice(None), slice(None)), "Capital Expenditure"].sum()
    opex = stats.loc[(slice(None), slice(None)), "Operational Expenditure"].sum()
    cost_breakdown[scenario] = {"CAPEX": capex, "OPEX": opex}

cost_breakdown["smr_only"]["CAPEX"] += TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_SMR
cost_breakdown["wind_bess_gas"]["CAPEX"] += TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS
cost_breakdown["wind_bess_gas_solar"]["CAPEX"] += TOTAL_ANNUITISED_INFRASTRUCTURE_COSTS_WITH_SOLAR
capex_list = [cost_breakdown[s]["CAPEX"] for s in scenarios_to_plot]
opex_list = [cost_breakdown[s]["OPEX"] for s in scenarios_to_plot]

# Plot stacked bar chart with total
fig = go.Figure(data=[
    go.Bar(
        name="CapEx",
        x=[scenario_labels[s] for s in scenarios_to_plot],
        y=capex_list,
        marker_color="#636EFA"
    ),
    go.Bar(
        name="OpEx",
        x=[scenario_labels[s] for s in scenarios_to_plot],
        y=opex_list,
        marker_color= "#00CC96"
    ),
])

fig.update_layout(
    barmode="stack",
    title="Annual Cost Breakdown by Scenario (Annuitised CapEx vs Yearly OpEx)",
    yaxis_title="Annual Cost (£)",
    xaxis_title="Scenario",
    template="plotly_white",
    legend_title_text="Cost Type"
)
fig.write_html(f"scenario=cost_breakdown_comparison_capex_opex_{DATA_CENTRE_LOAD_MW}MW.html")
fig.show()

In [ ]:
total_gas_mwh_usage = n.generators_t.p["gas_turbine"].sum()
total_data_centre_mwh_usage = n.loads_t.p["data_centre_load"].sum()
print(
    f"Ratio of data centre load covered by gas turbine: {total_gas_mwh_usage / total_data_centre_mwh_usage:.2%}"
)
CARBON_EMISSION_GAS_PER_KWH = 0.184 # https://questions-statements.parliament.uk/written-questions/detail/2015-11-26/17799#:~:text=Technology%20%2F%20Fuel,0.497
KG_TO_TONNES = 0.001
carbon_emissions_annual = (
    total_gas_mwh_usage * (CARBON_EMISSION_GAS_PER_KWH * KW_TO_MW) * KG_TO_TONNES
)
print(f"Annual Carbon Emissions from Gas Turbine: {carbon_emissions_annual:,.2f} tCO2")
